<a href="https://colab.research.google.com/github/amitsalunkecs/google-collab-proj/blob/main/Copy_of_Getting_started_with_google_colab_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab is making it easier than ever to integrate powerful Generative AI capabilities into your projects. We are launching public preview for a simple and intuitive Python library (google.colab.ai) to access state-of-the-art language models directly within Colab environments. All users have free access to most popular LLMs, while paid users have access to a wider selection of models. This means users can spend less time on configuration and set up and more time bringing their ideas to life. With just a few lines of code, you can now perform a variety of tasks:
- Generate text
- Translate languages
- Write creative content
- Categorize text

Happy Coding!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/Getting_started_with_google_colab_ai.ipynb)

In [ ]:
# @title List available models
from google.colab import ai

ai.list_models()

In [ ]:
#Defining the Search Tool
!pip install -U langchain_openai
!pip install -U langchain_google_genai
!pip install -U langchain_community
!pip install -U langchain-google-community

# Colab's userdata module
from google.colab import userdata
import os

os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_CSE_ID'] = userdata.get('GOOGLE_CSE_ID')
os.environ['SERP_API_KEY'] = userdata.get('SERPER_KEY')

#Defining the Search Tool
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

def top5_results(query):
    return search.results(query, 5)

search_tool = Tool(
    name="Google Search",
    description="Searches Google for recent results.",
    func=top5_results,
)

#Chaining the Model with a Prompt

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
model.bind_tools(tools=[search_tool])

prompt = PromptTemplate(
    template="Please Search this {topic} and give me a precise accurate result.",
    input_variables=["topic"]
)

chain = prompt | model | StrOutputParser()

result = chain.invoke({'topic': 'When Bangladesh was born? with information link'})
print(result)


#Structured Output with Pydantic

from pydantic import BaseModel, Field
from typing import Optional

class WebResult(BaseModel):
    web_result: str = Field(..., description="The result of the web page.")
    link: Optional[str] = Field(default=None, description="The link to the web page.")

structured_model_pydantic = model.with_structured_output(WebResult)

prompt = PromptTemplate(
    template="Please make a report on {topic} ",
    input_variables=["topic"]
)

chain = prompt | structured_model_pydantic

result = chain.invoke({'topic': 'why coldwar did happen?'})
print(result)
